In [1]:

class SpiderCroppedVideoReader():
    """ This class mimics OpenCV's VideoCapture class, but instead reads
        the *.ufmf files, which contain 200x200 cropped/rotated and monochrome
        spider images.
    """

    def __init__(self, filename, progressCallback=None):
        self.fname = filename
        self.iframe = 0
        self.progressCallback = progressCallback

        # DEBUG
        self.MAX_NUM_FRAMES = 50 * 3600 * 24 * 100

        # Is this a supported input file?
        if not self.fname.endswith('.ufmf'):
            raise Exception('Unsupported input file. Only *.ufmf files allowed.')

        # Open file to determine size
        from motmot.SpiderMovie import SpiderMovie
        mov = SpiderMovie(filename)
        #self.arr = np.memmap(self.fname, dtype=np.uint8, mode='r')
        #N = self.arr.shape[0]
        N = mov.shape[0]*mov.shape[1]*mov.shape[2]
        #del self.arr
        # Ensure this file has the right size
        if N % (1024 * 1024) != 0:
            raise Exception('Image does not have expected size of 1024x1024.')
        # Open again with right shape
        #self.arr = np.memmap(self.fname, dtype=np.uint8, mode='r', shape=(int(N / (1024 * 1024)), 1024, 1024))
        self.matrix_shape = mov.shape
        self.mov = mov

    # Get various metadata
    def get(self, i):
        # Call progress callback
        if self.progressCallback is not None:
            try:
                self.progressCallback(float(i) / self.matrix_shape[0])
            except:
                pass
        # NUM. FRAMES
        if i == 7:
            if self.matrix_shape is not None:
                return min(self.MAX_NUM_FRAMES, self.matrix_shape[0])
            else:
                raise Exception('File closed.')
        # FPS
        elif i == 5:
            return 50
        # HEIGHT
        elif i == 4:
            return 1024
        # WIDTH
        elif i == 3:
            return 1024
        # ERROR
        else:
            raise Exception('Unsupported metadata requested: {}'.format(i))

    def isOpened(self):
        return True

    def read(self):
        if self.iframe >= self.get(7):
            return False, np.zeros((1024, 1024), dtype=np.uint8)
        else:
            self.iframe += 1
            return True, self.mov[self.iframe - 1]

In [2]:
basepath = 'Z:/HsinYi/Test Video From Darya/test_clip_dlc/'
project_name = 'TEST'
scorer = 'PC'

import os, glob


video_list = glob.glob(os.path.join(basepath, 'raw/')+'*.s.ufmf')
video_list


['Z:/HsinYi/Test Video From Darya/test_clip_dlc/raw\\bias_video_cam_2_date_2023_05_31_time_19_56_23_v001.all.s.ufmf']

In [3]:
import deeplabcut
deeplabcut.create_new_project(project_name , scorer, video_list, working_directory=basepath, copy_videos= True, videoReader = SpiderCroppedVideoReader)

Created "\\10.99.66.32\Team Spider\HsinYi\Test Video From Darya\test_clip_dlc\TEST-PC-2023-09-24\videos"
Created "\\10.99.66.32\Team Spider\HsinYi\Test Video From Darya\test_clip_dlc\TEST-PC-2023-09-24\labeled-data"
Created "\\10.99.66.32\Team Spider\HsinYi\Test Video From Darya\test_clip_dlc\TEST-PC-2023-09-24\training-datasets"
Created "\\10.99.66.32\Team Spider\HsinYi\Test Video From Darya\test_clip_dlc\TEST-PC-2023-09-24\dlc-models"
Copying the videos
\\10.99.66.32\Team Spider\HsinYi\Test Video From Darya\test_clip_dlc\TEST-PC-2023-09-24\videos\bias_video_cam_2_date_2023_05_31_time_19_56_23_v001.all.s.ufmf
Generated "\\10.99.66.32\Team Spider\HsinYi\Test Video From Darya\test_clip_dlc\TEST-PC-2023-09-24\config.yaml"

A new project with name TEST-PC-2023-09-24 is created at \\10.99.66.32\Team Spider\HsinYi\Test Video From Darya\test_clip_dlc and a configurable file (config.yaml) is stored there. Change the parameters in this file to adapt to your project's needs.
 Once you have chan

c:\users\gordus_lab\documents\repositories\deeplabcut\deeplabcut\motmot\ufmf\ufmf.py:199: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  assert key not in value


'\\\\10.99.66.32\\Team Spider\\HsinYi\\Test Video From Darya\\test_clip_dlc\\TEST-PC-2023-09-24\\config.yaml'

In [4]:
config_path = basepath+ '/TEST-PC-2023-09-24/config.yaml'
cfg = deeplabcut.auxiliaryfunctions.read_config(config_path)
cfg['bodyparts'] =['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25']
deeplabcut.auxiliaryfunctions.write_config(config_path, cfg)

In [5]:
## Skip extract frames
## Skip label frames

## Generate labeling csv
import os, glob

video_list = glob.glob(os.path.join(basepath, 'raw/')+'*.s.ufmf')
videoname = video_list[0].split('raw\\')[1]
joint_filename = basepath + '/croprot/' +'bias_video_cam_2_date_2023_05_31_time_19_56_23_v001.all.s_dlc_abs.npy'

from deeplabcut.generate_training_dataset import creating_labeling_csv
creating_labeling_csv.creating_labeling_csv(config_path, basepath, videoname, joint_filename)


CREATING-SOME LABELS FOR THE FRAMES


In [6]:
deeplabcut.check_labels(config_path, basepath=basepath)

Creating images with labels by PC.
They are stored in the following folder: \\10.99.66.32\Team Spider\HsinYi\Test Video From Darya\test_clip_dlc\TEST-PC-2023-09-24\labeled-data\bias_video_cam_2_date_2023_05_31_time_19_56_23_v001.all.s.ufmf_labeled.
If all the labels are ok, then use the function 'create_training_dataset' to create the training dataset!


In [7]:
print("CREATING TRAININGSET")
deeplabcut.create_training_dataset(config_path)

CREATING TRAININGSET
0/28914
1/28914
2/28914


C:\Users\Gordus_Lab\anaconda3\envs\dlc-windowsCPU-trainvideo\lib\site-packages\deeplabcut\generate_training_dataset\trainingsetmanipulation.py:612: RuntimeWarning: invalid value encountered in less
  if Data[bodypart]['x'][jj]<np.shape(im)[1] and Data[bodypart]['y'][jj]<np.shape(im)[0]: #are labels in image?


3/28914
4/28914
5/28914
6/28914
7/28914
8/28914
9/28914
10/28914
11/28914
12/28914
13/28914
14/28914
15/28914
16/28914
17/28914
18/28914
19/28914
20/28914
21/28914
22/28914
23/28914
24/28914
25/28914
26/28914
27/28914
28/28914
29/28914
30/28914
31/28914
32/28914
33/28914
34/28914
35/28914
36/28914
37/28914
38/28914
39/28914
40/28914
41/28914
42/28914
43/28914
44/28914
45/28914
46/28914
47/28914
48/28914
49/28914
50/28914
51/28914
52/28914
53/28914
54/28914
55/28914
56/28914
57/28914
58/28914
59/28914
60/28914
61/28914
62/28914
63/28914
64/28914
65/28914
66/28914
67/28914
68/28914
69/28914
70/28914
71/28914
72/28914
73/28914
74/28914
75/28914
76/28914
77/28914
78/28914
79/28914
80/28914
81/28914
82/28914
83/28914
84/28914
85/28914
86/28914
87/28914
88/28914
89/28914
90/28914
91/28914
92/28914
93/28914
94/28914
95/28914
96/28914
97/28914
98/28914
99/28914
100/28914
101/28914
102/28914
103/28914
104/28914
105/28914
106/28914
107/28914
108/28914
109/28914
110/28914
111/28914
112/28914
113/

C:\Users\Gordus_Lab\anaconda3\envs\dlc-windowsCPU-trainvideo\lib\site-packages\deeplabcut\generate_training_dataset\trainingsetmanipulation.py:324: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  docs.append(yaml.load(raw_doc))


In [ ]:
## 
import deeplabcut
basepath = 'Z:/HsinYi/Test Video From Darya/test_clip_dlc/'
config_path = basepath+ '/TEST-PC-2023-09-24/config.yaml'
print('Train the network')
deeplabcut.train_network(config_path)

C:\Users\Gordus_Lab\anaconda3\envs\dlc-windowsCPU-trainvideo\lib\site-packages\deeplabcut\pose_estimation_tensorflow\config.py:43: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  yaml_cfg = edict(yaml.load(f))
Config:
{'Task': None,
 'TrainingFraction': None,
 'all_joints': [[0],
                [1],
                [2],
                [3],
                [4],
                [5],
                [6],
                [7],
                [8],
                [9],
                [10],
                [11],
                [12],
                [13],
                [14],
                [15],
                [16],
                [17],
                [18],
                [19],
                [20],
                [21],
                [22],
                [23],
                [24],
                [25]],
 'all_joints_names': ['0',
                      '1',
       

Train the network


c:\users\gordus_lab\documents\repositories\deeplabcut\deeplabcut\motmot\ufmf\ufmf.py:199: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  assert key not in value


INFO:tensorflow:Restoring parameters from C:\Users\Gordus_Lab\anaconda3\envs\dlc-windowsCPU-trainvideo\lib\site-packages\deeplabcut\pose_estimation_tensorflow\models\pretrained\resnet_v1_50.ckpt


Restoring parameters from C:\Users\Gordus_Lab\anaconda3\envs\dlc-windowsCPU-trainvideo\lib\site-packages\deeplabcut\pose_estimation_tensorflow\models\pretrained\resnet_v1_50.ckpt


Training parameter:
{'stride': 8.0, 'weigh_part_predictions': False, 'weigh_negatives': False, 'fg_fraction': 0.25, 'weigh_only_present_joints': False, 'mean_pixel': [123.68, 116.779, 103.939], 'shuffle': True, 'snapshot_prefix': 'Z:\\HsinYi\\Test Video From Darya\\test_clip_dlc\\TEST-PC-2023-09-24\\dlc-models\\iteration-0\\TESTSep24-trainset95shuffle1\\train\\snapshot', 'log_dir': 'log', 'global_scale': 0.8, 'location_refinement': True, 'locref_stdev': 7.2801, 'locref_loss_weight': 0.05, 'locref_huber_loss': True, 'optimizer': 'sgd', 'intermediate_supervision': False, 'intermediate_supervision_layer': 12, 'regularize': False, 'weight_decay': 0.0001, 'mirror': False, 'crop_pad': 0, 'scoremap_dir': 'test', 'dataset_type': 'default', 'use_gt_segm': False, 'batch_size': 1, 'video': False, 'video_batch': False, 'crop': True, 'cropratio': 0.4, 'minsize': 100, 'leftwidth': 400, 'rightwidth': 400, 'topheight': 400, 'bottomheight': 400, 'Task': None, 'scorer': None, 'date': None, 'multianimalp

  0%|                                                                        | 993/1030001 [42:13<908:39:13,  3.18s/it]